# Polars

In [ ]:
pip install polars pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=a47b276ca02dddd3c3e1fd6fbe187d7bb38a5f1f87f5aaa1d9e41c733dbb262c
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean, stddev
import pandas as pd
import polars as pl
import dask.dataframe as dd
import numpy as np
import time

# Configuración de Spark
spark = SparkSession.builder.appName("Comparativa").getOrCreate()

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
# Simulación de datos
np.random.seed(42)
data = np.random.rand(100_000_000)
pdf = pd.DataFrame({'data': data})

# DataFrame de Spark
#sdf = spark.createDataFrame(pdf)

# Dask
ddf = dd.from_pandas(pdf, npartitions=100)

# Polars
pl_df = pl.DataFrame({'data': data})

# Tiempos de ejecución y estadísticas
times = {}

# Operaciones con PySpark
#start_time_spark = time.time()
#mean_val_spark = sdf.select(mean('data')).collect()[0][0]
#median_val_spark = sdf.approxQuantile('data', [0.5], 0.01)[0]  # Aproximado
#std_dev_spark = sdf.select(stddev('data')).collect()[0][0]
#times['PySpark'] = time.time() - start_time_spark, mean_val_spark, median_val_spark, std_dev_spark

# Tiempo Dask
start_time_dask = time.time()
mean_val_dask = ddf['data'].mean().compute()
median_val_dask = ddf['data'].quantile(0.5).compute()
std_dev_dask = ddf['data'].std().compute()
times['Dask'] = time.time() - start_time_dask, mean_val_dask, median_val_dask, std_dev_dask

# Tiempo Polars
start_time_pl = time.time()
mean_val_pl = pl_df.select(pl.mean('data')).to_numpy()[0][0]
median_val_pl = pl_df.select(pl.median('data')).to_numpy()[0][0]
std_dev_pl = pl_df.select(pl.std('data')).to_numpy()[0][0]
times['Polars'] = time.time() - start_time_pl, mean_val_pl, median_val_pl, std_dev_pl

# Tiempo pandas
start_time_pd = time.time()
mean_val_pd = pdf['data'].mean()
median_val_pd = pdf['data'].median()
std_dev_pd = pdf['data'].std()
times['pandas'] = time.time() - start_time_pd, mean_val_pd, median_val_pd, std_dev_pd

# Tiempo NumPy
start_time_np = time.time()
mean_val_np = np.mean(data)
median_val_np = np.median(data)
std_dev_np = np.std(data)
times['NumPy'] = time.time() - start_time_np, mean_val_np, median_val_np, std_dev_np

# Calcular la herramienta más rápida
fastest_tool = min(times, key=lambda x: times[x][0])
fastest_time = times[fastest_tool][0]

# Calcular cuántas veces es más rápida la herramienta más rápida que las demás
speedups = {tool: fastest_time / time_taken[0] for tool, time_taken in times.items()}

# Imprimir resultados
print("Tiempos de ejecución y estadísticas:")
for tool, (time_taken, mean_val, median_val, std_dev) in times.items():
    print(f"{tool}: Tiempo: {time_taken:.4f} segundos, Media: {mean_val:.6f}, Mediana: {median_val:.6f}, Desv. Est.: {std_dev:.6f}")

print(f"\nLa herramienta más rápida fue {fastest_tool} con {fastest_time:.4f} segundos.")

Tiempos de ejecución y estadísticas:
Dask: Tiempo: 5.1864 segundos, Media: 0.499980, Mediana: 0.501418, Desv. Est.: 0.288651
Polars: Tiempo: 2.0756 segundos, Media: 0.499980, Mediana: 0.499948, Desv. Est.: 0.288651
pandas: Tiempo: 5.7083 segundos, Media: 0.499980, Mediana: 0.499948, Desv. Est.: 0.288651
NumPy: Tiempo: 2.3448 segundos, Media: 0.499980, Mediana: 0.499948, Desv. Est.: 0.288651

La herramienta más rápida fue Polars con 2.0756 segundos.


In [ ]:
import pandas as pd
import numpy as np
import polars as pl

# Configuración para la generación de datos
num_rows = 1000
categories = ['A', 'B', 'C', 'D']

# Generar datos simulados
np.random.seed(42)  # Para reproducibilidad
df = pd.DataFrame({
    'Category': np.random.choice(categories, num_rows),
    'Num1': np.random.rand(num_rows),
    'Num2': np.random.rand(num_rows) * 100,
    'Num3': np.random.randn(num_rows) * 50,
    'Num4': np.random.rand(num_rows) * 1000,
    'Num5': np.random.randint(0, 100, num_rows)
})

# Escribir los datos a un archivo .csv
csv_file_path = 'simulated_data.csv'
df.to_csv(csv_file_path, index=False)

# Leer el archivo .csv con Polars
df_polars = pl.read_csv(csv_file_path)

# Mostrar el DataFrame de Polars
print(df_polars)

shape: (1_000, 6)
┌──────────┬──────────┬───────────┬────────────┬────────────┬──────┐
│ Category ┆ Num1     ┆ Num2      ┆ Num3       ┆ Num4       ┆ Num5 │
│ ---      ┆ ---      ┆ ---       ┆ ---        ┆ ---        ┆ ---  │
│ str      ┆ f64      ┆ f64       ┆ f64        ┆ f64        ┆ i64  │
╞══════════╪══════════╪═══════════╪════════════╪════════════╪══════╡
│ C        ┆ 0.698162 ┆ 51.908179 ┆ 26.056121  ┆ 184.632457 ┆ 29   │
│ D        ┆ 0.536096 ┆ 47.918188 ┆ 32.260779  ┆ 342.107621 ┆ 97   │
│ A        ┆ 0.309528 ┆ 2.564207  ┆ 27.780223  ┆ 429.95762  ┆ 57   │
│ C        ┆ 0.813795 ┆ 34.124783 ┆ 4.479034   ┆ 830.949171 ┆ 67   │
│ …        ┆ …        ┆ …         ┆ …          ┆ …          ┆ …    │
│ A        ┆ 0.61949  ┆ 57.727903 ┆ 56.781007  ┆ 675.586774 ┆ 85   │
│ D        ┆ 0.463494 ┆ 86.557715 ┆ -55.314676 ┆ 508.89407  ┆ 26   │
│ D        ┆ 0.379786 ┆ 98.073934 ┆ -41.225708 ┆ 861.445686 ┆ 82   │
│ C        ┆ 0.863334 ┆ 40.758421 ┆ -30.429486 ┆ 866.035675 ┆ 75   │
└──────────┴────

In [ ]:
pip install sqlalchemy connectorx polars adbc-driver-sqlite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 MB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.8/969.8 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 52.7 MB/s eta 0:00:00


In [ ]:
import sqlite3
import numpy as np
import pandas as pd

# Conexión (o creación si no existe) a la base de datos SQLite
conn = sqlite3.connect('test.db')

# Generar datos simulados
np.random.seed(0)  # Para reproducibilidad
num_rows = 1000
categories = ['A', 'B', 'C', 'D']  # Datos categóricos
data = {
    'Category': np.random.choice(categories, num_rows),
    'Num1': np.random.rand(num_rows),  # Distribución uniforme
    'Num2': np.random.randn(num_rows) * 10 + 50,  # Distribución normal
    'Num3': np.log(np.random.rand(num_rows) * 10 + 1),  # Distribución logarítmica
    'Num4': np.random.exponential(1, num_rows),  # Distribución exponencial
    'Num5': np.random.poisson(5, num_rows)  # Distribución de Poisson
}

df = pd.DataFrame(data)

# Crear tabla y insertar los datos en SQLite
df.to_sql('simulated_table', conn, if_exists='replace', index=False)

# Cerrar conexión a la base de datos
conn.close()

In [ ]:
import polars as pl
from sqlalchemy import create_engine

# Crear el motor de conexión para SQLite
engine = create_engine('sqlite:///test.db')

# Consulta SQL para leer los datos
query = "SELECT * FROM simulated_table"

# Utilizar pl.read_database para leer los datos en un DataFrame de Polars
df_polars = pl.read_database(query=query, connection=engine.connect())

# Mostrar el DataFrame de Polars
print(df_polars)

shape: (1_000, 6)
┌──────────┬──────────┬───────────┬──────────┬──────────┬──────┐
│ Category ┆ Num1     ┆ Num2      ┆ Num3     ┆ Num4     ┆ Num5 │
│ ---      ┆ ---      ┆ ---       ┆ ---      ┆ ---      ┆ ---  │
│ str      ┆ f64      ┆ f64       ┆ f64      ┆ f64      ┆ i64  │
╞══════════╪══════════╪═══════════╪══════════╪══════════╪══════╡
│ A        ┆ 0.310381 ┆ 62.21385  ┆ 0.918714 ┆ 0.259572 ┆ 3    │
│ D        ┆ 0.373035 ┆ 48.071582 ┆ 1.937057 ┆ 1.815245 ┆ 3    │
│ B        ┆ 0.52497  ┆ 49.666807 ┆ 1.852578 ┆ 0.419246 ┆ 2    │
│ A        ┆ 0.750595 ┆ 34.691965 ┆ 1.093287 ┆ 3.87489  ┆ 5    │
│ …        ┆ …        ┆ …         ┆ …        ┆ …        ┆ …    │
│ A        ┆ 0.193623 ┆ 26.278061 ┆ 1.470373 ┆ 0.387993 ┆ 6    │
│ C        ┆ 0.11225  ┆ 36.18255  ┆ 1.445977 ┆ 1.159704 ┆ 3    │
│ B        ┆ 0.042364 ┆ 48.875562 ┆ 2.394008 ┆ 0.144141 ┆ 5    │
│ A        ┆ 0.227741 ┆ 58.978642 ┆ 2.353668 ┆ 1.635521 ┆ 2    │
└──────────┴──────────┴───────────┴──────────┴──────────┴──────┘


### Paso 1: Creación de Datos Simulados

Primero, vamos a crear un DataFrame con Polars que contenga una columna categórica (`Category`) y cinco columnas numéricas (`Num1` a `Num5`) con diferentes tipos de datos numéricos.

```python
import polars as pl
import numpy as np

# Establecer la semilla para la reproducibilidad
np.random.seed(0)

# Crear el DataFrame
df = pl.DataFrame({
    "Category": np.random.choice(["A", "B", "C", "D"], 1000),
    "Num1": np.random.rand(1000) * 100,
    "Num2": np.random.randn(1000),
    "Num3": np.random.randint(1, 100, 1000),
    "Num4": np.random.exponential(1, 1000),
    "Num5": np.random.rand(1000) * 50
})

print(df)
```



In [ ]:
import polars as pl
import numpy as np

# Establecer la semilla para la reproducibilidad
np.random.seed(0)

# Crear el DataFrame
df = pl.DataFrame({
    "Category": np.random.choice(["A", "B", "C", "D"], 1000),
    "Num1": np.random.rand(1000) * 100,
    "Num2": np.random.randn(1000),
    "Num3": np.random.randint(1, 100, 1000),
    "Num4": np.random.exponential(1, 1000),
    "Num5": np.random.rand(1000) * 50
})

print(df)

shape: (1_000, 6)
┌──────────┬───────────┬───────────┬──────┬──────────┬───────────┐
│ Category ┆ Num1      ┆ Num2      ┆ Num3 ┆ Num4     ┆ Num5      │
│ ---      ┆ ---       ┆ ---       ┆ ---  ┆ ---      ┆ ---       │
│ str      ┆ f64       ┆ f64       ┆ i64  ┆ f64      ┆ f64       │
╞══════════╪═══════════╪═══════════╪══════╪══════════╪═══════════╡
│ A        ┆ 31.038083 ┆ 1.221385  ┆ 93   ┆ 0.327546 ┆ 25.425978 │
│ D        ┆ 37.303486 ┆ -0.192842 ┆ 66   ┆ 0.638946 ┆ 47.03379  │
│ B        ┆ 52.497044 ┆ -0.033319 ┆ 42   ┆ 1.280853 ┆ 43.510583 │
│ A        ┆ 75.059502 ┆ -1.530803 ┆ 39   ┆ 0.763945 ┆ 10.856792 │
│ …        ┆ …         ┆ …         ┆ …    ┆ …        ┆ …         │
│ A        ┆ 19.362329 ┆ -2.372194 ┆ 29   ┆ 1.178957 ┆ 4.428173  │
│ C        ┆ 11.224999 ┆ -1.381745 ┆ 1    ┆ 2.044436 ┆ 7.436232  │
│ B        ┆ 4.236405  ┆ -0.112444 ┆ 70   ┆ 0.26253  ┆ 7.77212   │
│ A        ┆ 22.774099 ┆ 0.897864  ┆ 58   ┆ 0.114053 ┆ 33.554484 │
└──────────┴───────────┴───────────┴──────┴─

### Ejemplo 1: Seleccionar Todas las Columnas del DataFrame

```python
all_columns = df.select(pl.all())
print(all_columns)
```

In [ ]:
all_columns = df.select(pl.all())
print(all_columns)

shape: (1_000, 6)
┌──────────┬───────────┬───────────┬──────┬──────────┬───────────┐
│ Category ┆ Num1      ┆ Num2      ┆ Num3 ┆ Num4     ┆ Num5      │
│ ---      ┆ ---       ┆ ---       ┆ ---  ┆ ---      ┆ ---       │
│ str      ┆ f64       ┆ f64       ┆ i64  ┆ f64      ┆ f64       │
╞══════════╪═══════════╪═══════════╪══════╪══════════╪═══════════╡
│ A        ┆ 31.038083 ┆ 1.221385  ┆ 93   ┆ 0.327546 ┆ 25.425978 │
│ D        ┆ 37.303486 ┆ -0.192842 ┆ 66   ┆ 0.638946 ┆ 47.03379  │
│ B        ┆ 52.497044 ┆ -0.033319 ┆ 42   ┆ 1.280853 ┆ 43.510583 │
│ A        ┆ 75.059502 ┆ -1.530803 ┆ 39   ┆ 0.763945 ┆ 10.856792 │
│ …        ┆ …         ┆ …         ┆ …    ┆ …        ┆ …         │
│ A        ┆ 19.362329 ┆ -2.372194 ┆ 29   ┆ 1.178957 ┆ 4.428173  │
│ C        ┆ 11.224999 ┆ -1.381745 ┆ 1    ┆ 2.044436 ┆ 7.436232  │
│ B        ┆ 4.236405  ┆ -0.112444 ┆ 70   ┆ 0.26253  ┆ 7.77212   │
│ A        ┆ 22.774099 ┆ 0.897864  ┆ 58   ┆ 0.114053 ┆ 33.554484 │
└──────────┴───────────┴───────────┴──────┴─

### Ejemplo 2: Seleccionar Columnas Específicas por Nombre

```python
specific_columns = df.select(["Category", "Num1", "Num3"])
print(specific_columns)
```

In [ ]:
specific_columns = df.select(["Category", "Num1", "Num3"])
print(specific_columns)

shape: (1_000, 3)
┌──────────┬───────────┬──────┐
│ Category ┆ Num1      ┆ Num3 │
│ ---      ┆ ---       ┆ ---  │
│ str      ┆ f64       ┆ i64  │
╞══════════╪═══════════╪══════╡
│ A        ┆ 31.038083 ┆ 93   │
│ D        ┆ 37.303486 ┆ 66   │
│ B        ┆ 52.497044 ┆ 42   │
│ A        ┆ 75.059502 ┆ 39   │
│ …        ┆ …         ┆ …    │
│ A        ┆ 19.362329 ┆ 29   │
│ C        ┆ 11.224999 ┆ 1    │
│ B        ┆ 4.236405  ┆ 70   │
│ A        ┆ 22.774099 ┆ 58   │
└──────────┴───────────┴──────┘


### Ejemplo 3: Seleccionar Columnas


```python
# En este ejemplo específico, sabemos que 'Num1' a 'Num5' son las columnas numéricas
numeric_columns = df.select([pl.col("Num1"), pl.col("Num2"), pl.col("Num3"), pl.col("Num4"), pl.col("Num5")])
print(numeric_columns)
```

In [ ]:
# En este ejemplo específico, sabemos que 'Num1' a 'Num5' son las columnas numéricas
numeric_columns = df.select([pl.col("Num1"), pl.col("Num2"), pl.col("Num3"), pl.col("Num4"), pl.col("Num5")])
print(numeric_columns)

shape: (1_000, 5)
┌───────────┬───────────┬──────┬──────────┬───────────┐
│ Num1      ┆ Num2      ┆ Num3 ┆ Num4     ┆ Num5      │
│ ---       ┆ ---       ┆ ---  ┆ ---      ┆ ---       │
│ f64       ┆ f64       ┆ i64  ┆ f64      ┆ f64       │
╞═══════════╪═══════════╪══════╪══════════╪═══════════╡
│ 31.038083 ┆ 1.221385  ┆ 93   ┆ 0.327546 ┆ 25.425978 │
│ 37.303486 ┆ -0.192842 ┆ 66   ┆ 0.638946 ┆ 47.03379  │
│ 52.497044 ┆ -0.033319 ┆ 42   ┆ 1.280853 ┆ 43.510583 │
│ 75.059502 ┆ -1.530803 ┆ 39   ┆ 0.763945 ┆ 10.856792 │
│ …         ┆ …         ┆ …    ┆ …        ┆ …         │
│ 19.362329 ┆ -2.372194 ┆ 29   ┆ 1.178957 ┆ 4.428173  │
│ 11.224999 ┆ -1.381745 ┆ 1    ┆ 2.044436 ┆ 7.436232  │
│ 4.236405  ┆ -0.112444 ┆ 70   ┆ 0.26253  ┆ 7.77212   │
│ 22.774099 ┆ 0.897864  ┆ 58   ┆ 0.114053 ┆ 33.554484 │
└───────────┴───────────┴──────┴──────────┴───────────┘



### FILTROS

Paso Adicional: Agregar Columna de Fechas



In [ ]:
import polars as pl
import numpy as np

# Generar fechas aleatorias
start_date = np.datetime64('2020-01-01')
end_date = np.datetime64('2020-12-31')
dates = np.arange(start_date, end_date, dtype='datetime64[D]')
random_dates = np.random.choice(dates, 1000)

# Crear DataFrame de Polars
df = pl.DataFrame({
    "Category": np.random.choice(["A", "B", "C", "D"], 1000),
    "Num1": np.random.rand(1000) * 100,
    "Num2": np.random.randn(1000),
    "Num3": np.random.randint(1, 100, 1000),
    "Num4": np.random.exponential(1, 1000),
    "Num5": np.random.rand(1000) * 50,
    "Date": random_dates
})

# Asegurarse de que la columna 'Date' es del tipo fecha
df = df.with_columns(df["Date"].cast(pl.Date))


### Ejemplo 1: Filtrar Filas Basadas en una Condición Simple

Vamos a filtrar las filas donde el valor de `Num1` sea mayor a 50.

```python
filtered_simple = df.filter(pl.col("Num1") > 50)
print(filtered_simple)
```



In [ ]:
filtered_simple = df.filter(pl.col("Num1") > 50)
print(filtered_simple)

shape: (498, 7)
┌──────────┬───────────┬───────────┬──────┬──────────┬───────────┬────────────┐
│ Category ┆ Num1      ┆ Num2      ┆ Num3 ┆ Num4     ┆ Num5      ┆ Date       │
│ ---      ┆ ---       ┆ ---       ┆ ---  ┆ ---      ┆ ---       ┆ ---        │
│ str      ┆ f64       ┆ f64       ┆ i64  ┆ f64      ┆ f64       ┆ date       │
╞══════════╪═══════════╪═══════════╪══════╪══════════╪═══════════╪════════════╡
│ D        ┆ 58.136755 ┆ 1.548457  ┆ 92   ┆ 0.723377 ┆ 17.847931 ┆ 2020-08-16 │
│ B        ┆ 63.965207 ┆ -0.005717 ┆ 96   ┆ 0.874726 ┆ 12.484402 ┆ 2020-04-08 │
│ B        ┆ 77.615439 ┆ 0.377103  ┆ 78   ┆ 1.69411  ┆ 15.93857  ┆ 2020-04-18 │
│ D        ┆ 97.777518 ┆ 0.2172    ┆ 17   ┆ 0.349406 ┆ 33.859185 ┆ 2020-04-02 │
│ …        ┆ …         ┆ …         ┆ …    ┆ …        ┆ …         ┆ …          │
│ C        ┆ 53.670425 ┆ -0.940735 ┆ 62   ┆ 1.992782 ┆ 12.922842 ┆ 2020-04-12 │
│ A        ┆ 82.10121  ┆ -0.343037 ┆ 60   ┆ 2.702458 ┆ 30.88326  ┆ 2020-10-04 │
│ C        ┆ 53.370971 ┆

### Ejemplo 2: Filtrar Utilizando Condiciones Compuestas

Ahora, filtraremos las filas que cumplan con dos condiciones: `Num1` mayor a 50 y `Num3` menor que 30.

```python
filtered_compound = df.filter((pl.col("Num1") > 50) & (pl.col("Num3") < 30))
print(filtered_compound)
```

In [ ]:
filtered_compound = df.filter((pl.col("Num1") > 50) & (pl.col("Num3") < 30))
print(filtered_compound)

shape: (133, 7)
┌──────────┬───────────┬───────────┬──────┬──────────┬───────────┬────────────┐
│ Category ┆ Num1      ┆ Num2      ┆ Num3 ┆ Num4     ┆ Num5      ┆ Date       │
│ ---      ┆ ---       ┆ ---       ┆ ---  ┆ ---      ┆ ---       ┆ ---        │
│ str      ┆ f64       ┆ f64       ┆ i64  ┆ f64      ┆ f64       ┆ date       │
╞══════════╪═══════════╪═══════════╪══════╪══════════╪═══════════╪════════════╡
│ D        ┆ 97.777518 ┆ 0.2172    ┆ 17   ┆ 0.349406 ┆ 33.859185 ┆ 2020-04-02 │
│ A        ┆ 74.174241 ┆ -0.351436 ┆ 3    ┆ 2.467354 ┆ 16.454902 ┆ 2020-01-16 │
│ D        ┆ 55.903986 ┆ -0.270658 ┆ 25   ┆ 0.588895 ┆ 5.132611  ┆ 2020-05-11 │
│ B        ┆ 89.146334 ┆ -2.498104 ┆ 16   ┆ 0.161631 ┆ 47.219254 ┆ 2020-03-07 │
│ …        ┆ …         ┆ …         ┆ …    ┆ …        ┆ …         ┆ …          │
│ A        ┆ 75.923623 ┆ 0.319232  ┆ 9    ┆ 1.234271 ┆ 37.479565 ┆ 2020-09-30 │
│ D        ┆ 53.753145 ┆ -0.700873 ┆ 28   ┆ 1.244588 ┆ 35.514753 ┆ 2020-04-13 │
│ B        ┆ 60.805498 ┆

### Ejemplo 3: Filtrar Fechas Dentro de un Rango Específico



In [ ]:
from datetime import datetime
import polars as pl

filtered_range_df = df.filter(
    pl.col("Date").is_between(pl.lit(datetime(2020, 3, 1)), pl.lit(datetime(2020, 6, 30)))
)

print(filtered_range_df)


shape: (329, 7)
┌──────────┬───────────┬───────────┬──────┬──────────┬───────────┬────────────┐
│ Category ┆ Num1      ┆ Num2      ┆ Num3 ┆ Num4     ┆ Num5      ┆ Date       │
│ ---      ┆ ---       ┆ ---       ┆ ---  ┆ ---      ┆ ---       ┆ ---        │
│ str      ┆ f64       ┆ f64       ┆ i64  ┆ f64      ┆ f64       ┆ date       │
╞══════════╪═══════════╪═══════════╪══════╪══════════╪═══════════╪════════════╡
│ B        ┆ 7.87384   ┆ -0.040114 ┆ 12   ┆ 0.302652 ┆ 47.156471 ┆ 2020-05-02 │
│ B        ┆ 63.965207 ┆ -0.005717 ┆ 96   ┆ 0.874726 ┆ 12.484402 ┆ 2020-04-08 │
│ B        ┆ 77.615439 ┆ 0.377103  ┆ 78   ┆ 1.69411  ┆ 15.93857  ┆ 2020-04-18 │
│ D        ┆ 97.777518 ┆ 0.2172    ┆ 17   ┆ 0.349406 ┆ 33.859185 ┆ 2020-04-02 │
│ …        ┆ …         ┆ …         ┆ …    ┆ …        ┆ …         ┆ …          │
│ C        ┆ 53.670425 ┆ -0.940735 ┆ 62   ┆ 1.992782 ┆ 12.922842 ┆ 2020-04-12 │
│ A        ┆ 11.794768 ┆ 0.926027  ┆ 35   ┆ 0.119729 ┆ 17.723049 ┆ 2020-05-30 │
│ D        ┆ 39.074001 ┆

Ejemplo: Añadir una Nueva Columna Calculada a Partir de Otras Columnas

Vamos a añadir una nueva columna llamada "Num6", que será el resultado de multiplicar "Num1" por "Num2".

In [ ]:
df_with_new_column = df.with_columns((pl.col("Num1") * pl.col("Num2")).alias("Num6"))
print(df_with_new_column)

shape: (1_000, 8)
┌──────────┬───────────┬───────────┬──────┬──────────┬───────────┬────────────┬────────────┐
│ Category ┆ Num1      ┆ Num2      ┆ Num3 ┆ Num4     ┆ Num5      ┆ Date       ┆ Num6       │
│ ---      ┆ ---       ┆ ---       ┆ ---  ┆ ---      ┆ ---       ┆ ---        ┆ ---        │
│ str      ┆ f64       ┆ f64       ┆ i64  ┆ f64      ┆ f64       ┆ date       ┆ f64        │
╞══════════╪═══════════╪═══════════╪══════╪══════════╪═══════════╪════════════╪════════════╡
│ A        ┆ 32.257447 ┆ -0.759242 ┆ 10   ┆ 0.64926  ┆ 15.228581 ┆ 2020-08-27 ┆ -24.491203 │
│ B        ┆ 7.87384   ┆ -0.040114 ┆ 12   ┆ 0.302652 ┆ 47.156471 ┆ 2020-05-02 ┆ -0.31585   │
│ A        ┆ 40.584596 ┆ 0.597292  ┆ 90   ┆ 0.877626 ┆ 33.292291 ┆ 2020-02-22 ┆ 24.240849  │
│ D        ┆ 16.762608 ┆ 0.255416  ┆ 88   ┆ 0.327104 ┆ 13.526529 ┆ 2020-12-29 ┆ 4.281438   │
│ …        ┆ …         ┆ …         ┆ …    ┆ …        ┆ …         ┆ …          ┆ …          │
│ C        ┆ 49.067502 ┆ -0.199424 ┆ 86   ┆ 0.696939

Ejemplo: Modificar Columnas Existentes Aplicando Transformaciones

Vamos a normalizar la columna "Num3" (ponerla en una escala de 0 a 1) utilizando el máximo y el mínimo de esa columna.

In [ ]:
df_modified = df.with_columns(
    ((pl.col("Num3") - pl.col("Num3").min()) / (pl.col("Num3").max() - pl.col("Num3").min())).alias("Num3_normalized")
)
print(df_modified)

shape: (1_000, 8)
┌──────────┬───────────┬───────────┬──────┬──────────┬───────────┬────────────┬─────────────────┐
│ Category ┆ Num1      ┆ Num2      ┆ Num3 ┆ Num4     ┆ Num5      ┆ Date       ┆ Num3_normalized │
│ ---      ┆ ---       ┆ ---       ┆ ---  ┆ ---      ┆ ---       ┆ ---        ┆ ---             │
│ str      ┆ f64       ┆ f64       ┆ i64  ┆ f64      ┆ f64       ┆ date       ┆ f64             │
╞══════════╪═══════════╪═══════════╪══════╪══════════╪═══════════╪════════════╪═════════════════╡
│ A        ┆ 32.257447 ┆ -0.759242 ┆ 10   ┆ 0.64926  ┆ 15.228581 ┆ 2020-08-27 ┆ 0.091837        │
│ B        ┆ 7.87384   ┆ -0.040114 ┆ 12   ┆ 0.302652 ┆ 47.156471 ┆ 2020-05-02 ┆ 0.112245        │
│ A        ┆ 40.584596 ┆ 0.597292  ┆ 90   ┆ 0.877626 ┆ 33.292291 ┆ 2020-02-22 ┆ 0.908163        │
│ D        ┆ 16.762608 ┆ 0.255416  ┆ 88   ┆ 0.327104 ┆ 13.526529 ┆ 2020-12-29 ┆ 0.887755        │
│ …        ┆ …         ┆ …         ┆ …    ┆ …        ┆ …         ┆ …          ┆ …               │
│ 

Ejemplo: Crear Columnas Indicadoras o Flags Basadas en Condiciones Específicas

Añadiremos una columna indicadora llamada "High_Num1" que será True si el valor de "Num1" es mayor que 30, y False en caso contrario.

In [ ]:
df_with_flag = df.with_columns(
    (pl.col("Num1") > 30).alias("High_Num1")
)
print(df_with_flag)

shape: (1_000, 8)
┌──────────┬───────────┬───────────┬──────┬──────────┬───────────┬────────────┬───────────┐
│ Category ┆ Num1      ┆ Num2      ┆ Num3 ┆ Num4     ┆ Num5      ┆ Date       ┆ High_Num1 │
│ ---      ┆ ---       ┆ ---       ┆ ---  ┆ ---      ┆ ---       ┆ ---        ┆ ---       │
│ str      ┆ f64       ┆ f64       ┆ i64  ┆ f64      ┆ f64       ┆ date       ┆ bool      │
╞══════════╪═══════════╪═══════════╪══════╪══════════╪═══════════╪════════════╪═══════════╡
│ A        ┆ 32.257447 ┆ -0.759242 ┆ 10   ┆ 0.64926  ┆ 15.228581 ┆ 2020-08-27 ┆ true      │
│ B        ┆ 7.87384   ┆ -0.040114 ┆ 12   ┆ 0.302652 ┆ 47.156471 ┆ 2020-05-02 ┆ false     │
│ A        ┆ 40.584596 ┆ 0.597292  ┆ 90   ┆ 0.877626 ┆ 33.292291 ┆ 2020-02-22 ┆ true      │
│ D        ┆ 16.762608 ┆ 0.255416  ┆ 88   ┆ 0.327104 ┆ 13.526529 ┆ 2020-12-29 ┆ false     │
│ …        ┆ …         ┆ …         ┆ …    ┆ …        ┆ …         ┆ …          ┆ …         │
│ C        ┆ 49.067502 ┆ -0.199424 ┆ 86   ┆ 0.696939 ┆ 15.7897

Ejemplo: Agrupar Datos por una o Varias Columnas y Calcular Estadísticas Resumidas

Agruparemos por la columna "Category" y calcularemos la suma, la media y el conteo para la columna "Num1".

In [ ]:
grouped_summary = df.group_by("Category").agg([
    pl.col("Num1").sum().alias("Sum_Num1"),
    pl.col("Num1").mean().alias("Mean_Num1"),
    pl.count("Num1").alias("Count_Num1")
])
print(grouped_summary)

shape: (4, 4)
┌──────────┬──────────────┬───────────┬────────────┐
│ Category ┆ Sum_Num1     ┆ Mean_Num1 ┆ Count_Num1 │
│ ---      ┆ ---          ┆ ---       ┆ ---        │
│ str      ┆ f64          ┆ f64       ┆ u32        │
╞══════════╪══════════════╪═══════════╪════════════╡
│ D        ┆ 10958.722653 ┆ 47.235874 ┆ 232        │
│ C        ┆ 12365.30705  ┆ 50.677488 ┆ 244        │
│ B        ┆ 12644.172709 ┆ 49.199116 ┆ 257        │
│ A        ┆ 13713.178445 ┆ 51.360219 ┆ 267        │
└──────────┴──────────────┴───────────┴────────────┘


Ejemplo: Agrupar y Aplicar Múltiples Agregaciones en una Sola Pasada

Agruparemos por la columna "Category" y aplicaremos múltiples agregaciones (suma y media) a varias columnas ("Num1", "Num2").

In [ ]:
multiple_aggregations = df.group_by("Category").agg([
    pl.col("Num1").sum().alias("Sum_Num1"),
    pl.col("Num1").mean().alias("Mean_Num1"),
    pl.col("Num2").sum().alias("Sum_Num2"),
    pl.col("Num2").mean().alias("Mean_Num2")
])
print(multiple_aggregations)

shape: (4, 5)
┌──────────┬──────────────┬───────────┬────────────┬───────────┐
│ Category ┆ Sum_Num1     ┆ Mean_Num1 ┆ Sum_Num2   ┆ Mean_Num2 │
│ ---      ┆ ---          ┆ ---       ┆ ---        ┆ ---       │
│ str      ┆ f64          ┆ f64       ┆ f64        ┆ f64       │
╞══════════╪══════════════╪═══════════╪════════════╪═══════════╡
│ D        ┆ 10958.722653 ┆ 47.235874 ┆ -36.980362 ┆ -0.159398 │
│ C        ┆ 12365.30705  ┆ 50.677488 ┆ 35.123323  ┆ 0.143948  │
│ B        ┆ 12644.172709 ┆ 49.199116 ┆ -4.333858  ┆ -0.016863 │
│ A        ┆ 13713.178445 ┆ 51.360219 ┆ 18.999462  ┆ 0.071159  │
└──────────┴──────────────┴───────────┴────────────┴───────────┘


Ejemplo: Realizar Agregaciones Condicionales Basadas en Valores de Otra Columna

Para este ejemplo, agruparemos por "Category" y calcularemos la suma de "Num1" solo para aquellos valores de "Num2" que sean mayores que 0.

In [ ]:
conditional_aggregation = df.group_by("Category").agg([
    pl.when(pl.col("Num2") > 0).then(pl.col("Num1")).sum().alias("Conditional_Sum_Num1")
])
print(conditional_aggregation)

shape: (4, 2)
┌──────────┬──────────────────────┐
│ Category ┆ Conditional_Sum_Num1 │
│ ---      ┆ ---                  │
│ str      ┆ f64                  │
╞══════════╪══════════════════════╡
│ A        ┆ 7654.056788          │
│ B        ┆ 6057.249019          │
│ D        ┆ 4901.528148          │
│ C        ┆ 7069.437766          │
└──────────┴──────────────────────┘


Ejemplo: Combinar Selección, Filtrado y Agregación

Imagina que queremos calcular la media de la columna "Num1" para los registros donde "Num3" sea mayor que 50, y además, solo para la categoría "B" en la columna "Category".

In [ ]:
# Primero, filtramos por la condición específica, luego seleccionamos la columna de interés,
# y finalmente calculamos la media.
mean_num1_for_category_B_and_num3_gt_50 = df.filter(
    (pl.col("Category") == "B") & (pl.col("Num3") > 50)
).select(
    pl.mean("Num1")
)

print(mean_num1_for_category_B_and_num3_gt_50)

shape: (1, 1)
┌───────────┐
│ Num1      │
│ ---       │
│ f64       │
╞═══════════╡
│ 51.233062 │
└───────────┘


Ejemplo: Uso de Expresiones para Crear Consultas Encadenadas Eficientes

Supongamos que queremos agregar una columna que indique si "Num2" está por encima de su media global y luego calcular la suma de "Num1" solo para esos casos, todo en una sola pasada.

In [ ]:
# Añadimos una columna indicadora para Num2 > mean(Num2),
# luego filtramos por esa condición y calculamos la suma de Num1.
sum_num1_where_num2_above_mean = df.with_columns(
    (pl.col("Num2") > pl.mean("Num2")).alias("Num2_above_mean")
).filter(
    pl.col("Num2_above_mean")
).select(
    pl.sum("Num1")
)

print(sum_num1_where_num2_above_mean)

shape: (1, 1)
┌──────────────┐
│ Num1         │
│ ---          │
│ f64          │
╞══════════════╡
│ 25533.264493 │
└──────────────┘


# JOINS

In [ ]:
import polars as pl

# DataFrame 1
df1 = pl.DataFrame({
    "id": [1, 2, 3, 4],
    "value_df1": ["A", "B", "C", "D"]
})

# DataFrame 2
df2 = pl.DataFrame({
    "id": [3, 4, 5, 6],
    "value_df2": ["E", "F", "G", "H"]
})

Ejemplo: Inner Join

El inner join devuelve las filas que tienen claves coincidentes en ambos DataFrames.

In [ ]:
inner_joined_df = df1.join(df2, on="id", how="inner")
print(inner_joined_df)

shape: (2, 3)
┌─────┬───────────┬───────────┐
│ id  ┆ value_df1 ┆ value_df2 │
│ --- ┆ ---       ┆ ---       │
│ i64 ┆ str       ┆ str       │
╞═════╪═══════════╪═══════════╡
│ 3   ┆ C         ┆ E         │
│ 4   ┆ D         ┆ F         │
└─────┴───────────┴───────────┘


Ejemplo: Left Join

El left join devuelve todas las filas del DataFrame izquierdo y las filas coincidentes del DataFrame derecho. Las filas del DataFrame izquierdo que no tienen coincidencias en el DataFrame derecho tienen valores null en las columnas del DataFrame derecho.

In [ ]:
left_joined_df = df1.join(df2, on="id", how="left")
print(left_joined_df)

shape: (4, 3)
┌─────┬───────────┬───────────┐
│ id  ┆ value_df1 ┆ value_df2 │
│ --- ┆ ---       ┆ ---       │
│ i64 ┆ str       ┆ str       │
╞═════╪═══════════╪═══════════╡
│ 1   ┆ A         ┆ null      │
│ 2   ┆ B         ┆ null      │
│ 3   ┆ C         ┆ E         │
│ 4   ┆ D         ┆ F         │
└─────┴───────────┴───────────┘


Simulación de Right Join

Dado que Polars no tiene un "right join" directo, invertimos el orden de los DataFrames y usamos un "left join".

In [ ]:
# Simulación de Right Join invirtiendo el orden y usando un left join
right_joined_df = df2.join(df1, on="id", how="left")
print(right_joined_df)

shape: (4, 3)
┌─────┬───────────┬───────────┐
│ id  ┆ value_df2 ┆ value_df1 │
│ --- ┆ ---       ┆ ---       │
│ i64 ┆ str       ┆ str       │
╞═════╪═══════════╪═══════════╡
│ 3   ┆ E         ┆ C         │
│ 4   ┆ F         ┆ D         │
│ 5   ┆ G         ┆ null      │
│ 6   ┆ H         ┆ null      │
└─────┴───────────┴───────────┘


Ejemplo: Outer Join

El outer join devuelve todas las filas de ambos DataFrames, con filas coincidentes de ambos lados donde estén disponibles. Si no hay coincidencia, el lado correspondiente tendrá valores null.

In [ ]:
outer_joined_df = df1.join(df2, on="id", how="outer")
print(outer_joined_df)

shape: (6, 4)
┌──────┬───────────┬──────────┬───────────┐
│ id   ┆ value_df1 ┆ id_right ┆ value_df2 │
│ ---  ┆ ---       ┆ ---      ┆ ---       │
│ i64  ┆ str       ┆ i64      ┆ str       │
╞══════╪═══════════╪══════════╪═══════════╡
│ 3    ┆ C         ┆ 3        ┆ E         │
│ 4    ┆ D         ┆ 4        ┆ F         │
│ null ┆ null      ┆ 5        ┆ G         │
│ null ┆ null      ┆ 6        ┆ H         │
│ 2    ┆ B         ┆ null     ┆ null      │
│ 1    ┆ A         ┆ null     ┆ null      │
└──────┴───────────┴──────────┴───────────┘


In [ ]:
import polars as pl

# DataFrame 1
df1 = pl.DataFrame({
    "id": [1, 2, 3, 4],
    "value_df": ["A", "B", "C", "D"]
})

# DataFrame 2
df2 = pl.DataFrame({
    "id": [3, 4, 5, 6],
    "value_df": ["E", "F", "G", "H"]
})

In [ ]:
# Anexión vertical
vertical_concatenated_df = pl.concat([df1, df2])
print(vertical_concatenated_df)

shape: (8, 2)
┌─────┬──────────┐
│ id  ┆ value_df │
│ --- ┆ ---      │
│ i64 ┆ str      │
╞═════╪══════════╡
│ 1   ┆ A        │
│ 2   ┆ B        │
│ 3   ┆ C        │
│ 4   ┆ D        │
│ 3   ┆ E        │
│ 4   ┆ F        │
│ 5   ┆ G        │
│ 6   ┆ H        │
└─────┴──────────┘
